In [1]:
import pandas as pd
import numpy as np

from read_record import Record,RecordReader
from scanning_window import scan_record
from collections import Counter

In [8]:
folder_path="D:/CPSC2021/Training_set_I"
file_name="data_25_15"

In [9]:
reading_record=RecordReader.read(folder_path,file_name,0,0,None)

In [10]:
# Get the keys of the object
keys=reading_record.__dict__.keys()

# Convert the keys to a list if you need
keys_list = list(keys)

# Print the keys
print(keys_list)

['_Record__parent', '_Record__signal', '_Record__symbol', '_Record__aux', '_Record__sample', '_Record__label', '_Record__sf', '_Record__collection']


In [ ]:
ecg_signal=reading_record._Record__signal
duration=len(ecg_signal)/reading_record._Record__sf
duration

In [ ]:
record_rhythm=reading_record._Record__aux
Counter(record_rhythm)

In [ ]:
record_label=reading_record._Record__label
record_label

In [ ]:
ecg_symbol=reading_record._Record__symbol
Counter(ecg_symbol)

In [ ]:
ecg_sample=reading_record._Record__sample
ecg_sample

In [ ]:
ecg_symbol[72]

In [ ]:
list(Counter(reading_record._Record__aux).keys())==['']

In [ ]:
get_interval=reading_record.get_valid_rhythm_interval(duration=30)
get_interval


In [ ]:
interval_signal=ecg_signal[get_interval[0][0]:get_interval[0][1]]
len(interval_signal)

In [11]:
from scanning_window import scan_record
test_scan=scan_record(reading_record,window_width=30)


There's rhythm annotation. dict_keys(['', '(AFL', '(N'])


NSR interval is from [(30111, 63367), (67703, 80908)]
AF interval is from []
30111
[234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251
 252 253 254 255 256 257 258 259 260 261 262 263 264 265]
left_end pt: 30361
[236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253
 254 255 256 257 258 259 260 261 262 263 264 265 266]
left_end pt: 30611
[237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254
 255 256 257 258 259 260 261 262 263 264 265 266 267 268]
left_end pt: 30861
[238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255
 256 257 258 259 260 261 262 263 264 265 266 267 268 269]
left_end pt: 31111
[239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256
 257 258 259 260 261 262 263 264 265 266 267 268 269 270]
left_end pt: 31361
[241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258
 259 260 261 262 263 264 265 266 267 268 269 270 271]
left_end pt: 31611
[242 243 244 245 246 247 

In [ ]:
a=[(2000,1000)]
?a

In [ ]:
a[0]